In [27]:
#pip install citipy - citypy already installed

In [28]:
# Import dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# Add citypy lbrary
from citipy import citipy

In [29]:
# Generate Lats range from -90 to 90
# Generate Lngs range from -180 to 180

In [30]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)

# Zip method combines the lats and lngs into a list of pairs. 
lat_lngs = zip(lats, lngs)
lat_lngs

In [31]:
# Add latitudes and longitudes to a list.
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [32]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

10